In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
import torch
import torchvision.transforms as T

# import some common libraries
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.data import build_detection_test_loader, DatasetMapper
from detectron2.evaluation import COCOEvaluator
from detectron2.config import get_cfg
from detectron2.data.datasets import register_coco_instances
from LossEvalHook import LossEvalHook

import detectron2.utils.comm as comm
import time
import datetime
import os

In [ ]:
class MyTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return COCOEvaluator(dataset_name, cfg, True, output_folder)
                     
    def build_hooks(self):
        hooks = super().build_hooks()
        hooks.insert(-1,LossEvalHook(
            cfg.TEST.EVAL_PERIOD,
            self.model,
            build_detection_test_loader(
                self.cfg,
                self.cfg.DATASETS.TEST[0],
                DatasetMapper(self.cfg,True)
            )
        ))
        return hooks

In [ ]:
dataset_location = "datasets/TEM-Nanoparticle-Recognition-12"
dataset_location = os.path.abspath(dataset_location)
output_path = dataset_location + "_output"

In [ ]:
print(dataset_location)

In [ ]:
# register training, test, and valid
register_coco_instances("trainingset", {}, f"{dataset_location}/train/_annotations.coco.json", f"{dataset_location}/train")
register_coco_instances("testset", {}, f"{dataset_location}/test/_annotations.coco.json", f"{dataset_location}/test")
register_coco_instances("validset", {}, f"{dataset_location}/valid/_annotations.coco.json", f"{dataset_location}/valid")

In [ ]:
cfg = get_cfg()
cfg.MODEL.DEVICE = "cuda"  # "cpu" or "cuda"

cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("trainingset",)
cfg.DATASETS.TEST = ("validset",)  # If you have a separate test dataset, you can specify it here
cfg.TEST.EVAL_PERIOD = 100
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 1000
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5  # Update with the number of classes in your dataset
cfg.OUTPUT_DIR = output_path  # Specify the directory to save trained models and logs

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = MyTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
# Define the path to save the config file
config_path = output_path + "/config.yaml"

# Save the configuration file
with open(config_path, "w") as f:
    f.write(cfg.dump())

In [ ]:
%load_ext tensorboard
%tensorboard --logdir output_path

In [ ]:
import json
import matplotlib.pyplot as plt

experiment_folder = 'datasets\TEM-Nanoparticle-Recognition-12_output'

def load_json_arr(json_path):
    lines = []
    with open(json_path, 'r') as f:
        for line in f:
            lines.append(json.loads(line))
    return lines

experiment_metrics = load_json_arr(experiment_folder + '/metrics.json')

plt.plot(
    [x['iteration'] for x in experiment_metrics if 'total_loss' in x], 
    [x['total_loss'] for x in experiment_metrics if 'total_loss' in x])
plt.plot(
    [x['iteration'] for x in experiment_metrics if 'validation_loss' in x], 
    [x['validation_loss'] for x in experiment_metrics if 'validation_loss' in x])
plt.legend(['total_loss', 'validation_loss'], loc='upper left')
plt.show()
print([x['iteration'] for x in experiment_metrics if 'total_loss' in x])